**Processamento da Trusted** - Desafio Parte III
- Carga de dados responsável pelo processamento dos arquivos JSON referente ao TMDB.

In [ ]:
!pip install boto3
!pip install pyspark

In [ ]:
import os
import boto3
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.appName("Trusted-CD").getOrCreate()

In [ ]:
aws_access_key_id = ''
aws_secret_access_key = ''
aws_session_token = ''

client = boto3.client('s3', aws_access_key_id = aws_access_key_id,
                  aws_secret_access_key = aws_secret_access_key,
                  aws_session_token = aws_session_token)

In [ ]:
dfs = []

bucket_name = "data-lake-livia"

entry_keys = ['Ram/tmdb/json/2024/02/24/data_53-215317.json', 'Ram/tmdb/json/2024/02/24/data_53-215442.json', 'Ram/tmdb/json/2024/02/24/data_53-215524.json', 'Ram/tmdb/json/2024/02/24/data_53-215706.json']

for aws_key in entry_keys:
  s3_object = client.get_object(Bucket=bucket_name, Key= aws_key)
  df_pd = pd.read_json(s3_object['Body'])
  dfs.append(df_pd)

df_pd = pd.concat(dfs, ignore_index=True)
print(df_pd)

In [ ]:
results = df_pd['results']
df_results = pd.DataFrame(results.tolist())
df_json = spark.createDataFrame(df_results)


In [ ]:
df_json.show()

df_json.printSchema()

In [ ]:
file_name = "suspense.parquet"

file_movies = f'/content/{file_name}'
df_json.coalesce(1).write.parquet(file_movies, mode="overwrite")

for filename in os.listdir(file_movies):
    if filename.startswith('part-'):
        new_filename = file_name
        old_filepath = os.path.join(file_movies, filename)
        new_filepath = os.path.join(file_movies, new_filename)
        os.rename(old_filepath, new_filepath)

In [ ]:
folder_name = "Suspense"

up_file_movies = f'/content/{file_name}/{file_name}'
aws_file_movies = f"TRT/{folder_name}/{file_name}"

try:
    client.upload_file(up_file_movies, bucket_name, aws_file_movies)
    print(f"Sucesso! Arquivo parquet enviado para o S3.")
except Exception as e:
    print(f"Erro durante o upload: {e}")